# Pre-work TikTok soundcharts API

## Setup

In [1]:
# Import modules
import pandas as pd

from datetime import datetime, timedelta
from soundcharts_API_client import *


In [2]:
# API connexion
x_app_id = 'PARIS-1_A28639D9'
x_api_key = '20bbdc4ab9ff0778'

sdch = SoundchartsClient(x_app_id, x_api_key)


## Get data

### Get TikTok charts

In [ ]:
# Start date
start_date = datetime.now()

# Get the data
results_df = []

for i in range (1):
    # Generate the date in ATOM format (YYYY-MM-DD)
    current_date = (start_date - timedelta(days=i)).strftime('%Y-%m-%dT%H:%M:%S+00:00')
    
    offset = 0
    limit = 100
    more_tracks = True
    
    while more_tracks:
        # Get the playlist data for the current date and offset
        response = sdch.get_playlist_tracklist_by_date(
            offset=offset, 
            limit=limit, 
            playlist_id='11e84493-4135-a14e-bca2-a0369fe50396', 
            date=current_date
        )
        
        # Extract playlist details
        playlist_details = response.get("related", {}).get("playlist", {})
        playlist_id = playlist_details.get("uuid")
        playlist_name = playlist_details.get("name")
        playlist_type = playlist_details.get("type")
        
        # Extract collection date
        collection_date = response.get("related", {}).get("date")
        
        # Extract track items and loop through them
        tracks = response.get("items", [])
        for item in tracks:
            track_position = item.get("position")
            track_name = item.get("song", {}).get("name")
            track_uuid = item.get("song", {}).get("uuid")
            
            # Append the track data to the list
            results_df.append({
                "playlist_id": playlist_id,
                "playlist_name": playlist_name,
                "collection_date": collection_date,
                "playlist_type": playlist_type,
                "track_position": track_position,
                "track_name": track_name,
                "track_uuid": track_uuid,
            })
        
        # Update the offset to fetch the next set of tracks
        offset += limit
        
        # If the number of returned items is less than the limit, stop the loop
        if len(tracks) < limit:
            more_tracks = False

# Convert the list to a DataFrame
df = pd.DataFrame(results_df)
df.to_excel('playlist_tracks.xlsx', index=False)

# Print the DataFrame
display(df)


### Get track and artist-level infos

In [3]:
# Import playlist data
df = pd.read_excel('playlist_tracks.xlsx')


In [4]:
# Get the tracks' and artists' info
results_df = []

for index, row in df.iterrows():
    track_id = row['track_uuid']
    
    playlist_id = row['playlist_id']
    playlist_name = row['playlist_name']
    collection_date = row['collection_date']
    playlist_type = row['playlist_type']
    track_position = row['track_position']

    tracks = sdch.get_track_metadata(track_id=track_id)

    track_names = tracks.get("object", {}).get("name")
    tracks_ids = tracks.get("object", {}).get("uuid")
    track_dates = tracks.get("object", {}).get("releaseDate")
    track_features = tracks.get("object", {}).get("audio")

    artist_names = []
    artist_ids = []

    for artist in tracks.get("object", {}).get("artists", []):
        artist_names.append(artist.get("name"))
        artist_ids.append(artist.get("uuid"))

    label_names = []
    label_types = []

    for label in tracks.get("object", {}).get("labels", []):
        label_names.append(label.get("name"))
        label_types.append(label.get("type"))
    
    track_genres = []
    
    for genre in tracks.get("object", {}).get("genres", []):
        track_genres.append(genre.get("root"))

    first_tracks = []
    artist_isnew = []

    for artist_id in artist_ids:
        artists_tracks_dates = sdch.get_first_track_date(artist_id=artist_id)
        
        first_date = None

        for item in artists_tracks_dates.get("items", []):
            release_date = item.get("releaseDate")
            if release_date and release_date != "null":
                first_date = release_date
                break 
        first_tracks.append(first_date if first_date else "No valid track date found")
        
        if first_date:
            first_date_dt = datetime.fromisoformat(first_date)
            collection_date_dt = datetime.fromisoformat(collection_date)
            if (collection_date_dt - first_date_dt).days > 365 * 2:
                new_artist = False
            else:
                new_artist = True
        artist_isnew.append(new_artist)


    results_df.append({
        "playlist_id": playlist_id,
        "playlist_name": playlist_name,
        "collection_date": collection_date,
        "playlist_type": playlist_type,
        "track_position": track_position,
        "track_name": track_names,
        "track_id": tracks_ids,
        "artist_name": artist_names,
        "artist_id": artist_ids,
        "artist_first_track": first_tracks,
        "artist_isnew": artist_isnew,
        "label_name": label_names,
        "label_type": label_types,
        "track_date": track_dates,
        "track_genres": track_genres,
        "track_features": track_features
    })

# Convert the list to a DataFrame
df = pd.DataFrame(results_df)
df.to_excel('tracks_info.xlsx', index=False)

# Print the DataFrame
display(df)


,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_id,artist_name,artist_id,artist_first_track,artist_isnew,label_name,label_type,track_date,track_genres,track_features
0,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"[J Balvin, Maluma, Feid, Karol G, Ovy on the D...","[11e81bca-6acb-9a46-8802-a0369fe50396, 11e81bc...","[1999-08-23T00:00:00+00:00, 1999-08-23T00:00:0...","[False, False, False, False, False, False, Fal...","[Interscope, Bichota]","[Universal, unknown]",2024-11-08T00:00:00+00:00,[latin],"{'danceability': 0.85, 'energy': 0.71, 'instru..."
1,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,2,High Road,e4b51982-7cf6-4fbb-8c70-a183f47dad26,[Zach Bryan],[2006fdf0-c896-11e9-8c83-801844edfa71],[2024-05-24T00:00:00+00:00],[True],"[Warner, Belting Bronco]","[Warner, Self released]",2024-11-07T00:00:00+00:00,[country],"{'danceability': 0.47, 'energy': 0.23, 'instru..."
2,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,3,Nice To Meet You,60ff68cf-86a9-4cc2-ba0d-3483e0c48485,[Myles Smith],[9ed290e6-c963-11e8-b7d5-549f35141000],[2017-09-25T00:00:00+00:00],[False],"[It's Okay, Feel]","[unknown, Other independent labels: ""Indie""]",2024-11-08T00:00:00+00:00,[folk & traditional],"{'danceability': 0.76, 'energy': 0.78, 'instru..."
3,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,4,last night's mascara,e93d01aa-e5fc-4144-8256-427bde5fd86b,[Griff],[11e81bc6-71e2-f878-b51d-a0369fe50396],[2015-02-06T00:00:00+00:00],[False],[Griff],[Self released],2024-11-08T00:00:00+00:00,[pop],"{'danceability': 0.71, 'energy': 0.7, 'instrum..."
4,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,5,Syrup Sandwiches (feat. Joey Bada$$),7d8defc4-dccb-4e17-b587-3506000829cf,"[Joey Bada$$, YBN Cordae]","[11e81bc7-a3c4-1d94-881e-a0369fe50396, 11e88ab...","[2012-03-23T00:00:00+00:00, 2018-02-23T00:00:0...","[False, False]",[Atlantic],[Warner],2024-11-15T00:00:00+00:00,[hip-hop & rap],"{'danceability': 0.49, 'energy': 0.68, 'instru..."


### Get playlist tracks' streams

In [5]:
# Import tracks playlist data
df = pd.read_excel('tracks_info.xlsx')


In [6]:
# Get the streams for each track
results_df = []

for index, row in df.iterrows():
    playlist_id = row['playlist_id']
    playlist_name = row['playlist_name']
    collection_date = row['collection_date']
    playlist_type = row['playlist_type']

    track_id = row['track_id']
    track_name = row['track_name']
    track_position = row['track_position']
    track_date = row['track_date']
    track_genres = row['track_genres']
    track_features = row['track_features']
    
    artist_name = row['artist_name']
    artist_id = row['artist_id']
    artist_first_track = row['artist_first_track']
    artist_isnew = row['artist_isnew']

    label_name = row['label_name']
    label_type = row['label_type']

    collection_date_dt = datetime.fromisoformat(collection_date)
    date_norm = collection_date_dt.strftime('%Y-%m-%d')

    streams_data = sdch.get_track_streams_by_date(
        startDate=date_norm,
        endDate=datetime.now().strftime('%Y-%m-%d'),
        track_id=track_id
    )

    processed_date = set()
    
    for item in streams_data.get("items", []):
        date = item.get("date")
        
        if date in processed_date:
            continue
        
        processed_date.add(date)
        
        first_plot = next(iter(item.get("plots", [])), None)
        
        if first_plot:
            results_df.append({
                "playlist_id": playlist_id,
                "playlist_name": playlist_name,
                "collection_date": collection_date,
                "playlist_type": playlist_type,
                "track_position": track_position,
                "track_name": track_name,
                "track_id": track_id,
                "artist_name": artist_name,
                "artist_id": artist_id,
                "artist_first_track": artist_first_track,
                "artist_isnew": artist_isnew,
                "label_name": label_name,
                "label_type": label_type,
                "track_date": track_date,
                "track_genres": track_genres,
                "track_features": track_features,
                "stream_date": date,
                "streams": first_plot.get("value")
            })

# Convert the list to a DataFrame
df = pd.DataFrame(results_df)
df.to_excel('tracks_info_streams.xlsx', index=False)

# Print the DataFrame
display(df)


,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_id,artist_name,artist_id,artist_first_track,artist_isnew,label_name,label_type,track_date,track_genres,track_features,stream_date,streams
0,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-21T00:00:00+00:00,31723699
1,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-20T00:00:00+00:00,29420845
2,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-19T00:00:00+00:00,27150515
3,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-18T00:00:00+00:00,25071499
4,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-17T00:00:00+00:00,22530887
5,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-16T00:00:00+00:00,19664576
6,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0b5891a19,"['J Balvin', 'Maluma', 'Feid', 'Karol G', 'Ovy...","['11e81bca-6acb-9a46-8802-a0369fe50396', '11e8...","['1999-08-23T00:00:00+00:00', '1999-08-23T00:0...","[False, False, False, False, False, False, Fal...","['Interscope', 'Bichota']","['Universal', 'unknown']",2024-11-08T00:00:00+00:00,['latin'],"{'danceability': 0.85, 'energy': 0.71, 'instru...",2024-11-15T00:00:00+00:00,16775627
7,11e84493-4135-a14e-bca2-a0369fe50396,New Music Friday,2024-11-13T11:06:59+00:00,Editorial,1,+57,be3c64a0-3000-40ec-a66e-c8f0